<a href="https://colab.research.google.com/github/sayakpaul/Denoised-Smoothing-TF/blob/main/Certification_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q easydict

In [3]:
import os
import urllib
from getpass import getpass

user = input('User name: ')
password = getpass('Password: ')
password = urllib.parse.quote(password)
repo_address = input('Repo Address: ')
branch_name = input('Branch name: ')

cmd_string = 'git clone https://{}:{}@github.com/{}.git -b {}'.format(
    user, password, repo_address, branch_name
)

os.system(cmd_string)
cmd_string, password = "", ""

User name: sayakpaul
Password: ··········
Repo Address: sayakpaul/Denoised-Smoothing-TF
Branch name: main


In [4]:
%%capture
!gsutil -m cp -r gs://denoised-smoothing-tf/denoiser_* .
!gsutil -m cp -r gs://denoised-smoothing-tf/resnet20_classifier .

In [5]:
import sys
sys.path.append("Denoised-Smoothing-TF")

from scripts import certify, analyze
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

tf.random.set_seed(42)
np.random.seed(42)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [6]:
print("Loading models.")
denoiser_mse = tf.keras.models.load_model("denoiser_mse")
denoiser_stab = tf.keras.models.load_model("denoiser_stability")
classifier = tf.keras.models.load_model("resnet20_classifier")
print("Models loaded.")

Loading models.
Models loaded.


In [7]:
(_, _), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_test = x_test.astype("float32") / 255.0
print(f"Total test examples: {len(x_test)}")

170500096/170498071 [==============================] - 23s 0us/step
Total test examples: 10000


In [8]:
certify.perform_certification_test(classifier, dataset=(x_test, y_test), sigma=0.25,
							   outfile="certification_output/sigma_0.25_no_denoiser")

idx	label	predict	radius	correct	time
0	[3]	2	0.115	0	0:00:19.542123
20	[7]	-1	0.0	0	0:00:02.748097
40	[4]	-1	0.0	0	0:00:02.723973
60	[7]	2	0.0509	0	0:00:02.706775
80	[8]	2	0.239	0	0:00:02.722867
100	[4]	3	0.147	0	0:00:02.724228
120	[8]	2	0.116	0	0:00:02.709119
140	[6]	2	0.000564	0	0:00:02.730524
160	[2]	2	0.0548	1	0:00:02.721210
180	[0]	2	0.11	0	0:00:02.768896
200	[5]	2	0.158	0	0:00:02.737095
220	[7]	2	0.305	0	0:00:02.744613
240	[1]	-1	0.0	0	0:00:02.721209
260	[8]	3	0.0401	0	0:00:02.762983
280	[9]	-1	0.0	0	0:00:02.716990
300	[6]	6	0.0764	1	0:00:02.732518
320	[3]	2	0.0234	0	0:00:02.758622
340	[2]	2	0.0719	1	0:00:02.759902
360	[9]	-1	0.0	0	0:00:02.761914
380	[6]	2	0.0401	0	0:00:02.769558
400	[9]	6	0.143	0	0:00:02.762646
420	[4]	2	0.0787	0	0:00:02.748421
440	[1]	6	0.0702	0	0:00:02.767536
460	[5]	2	0.256	0	0:00:02.777226
480	[8]	-1	0.0	0	0:00:02.812852
500	[4]	2	0.226	0	0:00:02.754199
520	[5]	2	0.0457	0	0:00:02.770533
540	[1]	-1	0.0	0	0:00:02.737404
560	[0]	2	0.0299	0	0:00:02.751923
580	[

In [9]:
denoiser_mse_classifier = tf.keras.Sequential([
    denoiser_mse,
    classifier
])
denoiser_stab_classifier = tf.keras.Sequential([
    denoiser_stab,
    classifier
])

In [10]:
certify.perform_certification_test(denoiser_mse_classifier, dataset=(x_test, y_test), sigma=0.25,
							   outfile="certification_output/sigma_0.25_denoiser_mse")

idx	label	predict	radius	correct	time
0	[3]	2	0.116	0	0:00:04.737114
20	[7]	-1	0.0	0	0:00:04.021654
40	[4]	-1	0.0	0	0:00:04.021092
60	[7]	2	0.0526	0	0:00:04.015941
80	[8]	2	0.25	0	0:00:04.070871
100	[4]	3	0.145	0	0:00:04.012033
120	[8]	2	0.109	0	0:00:03.997192
140	[6]	2	0.000564	0	0:00:04.024827
160	[2]	2	0.0578	1	0:00:04.000920
180	[0]	2	0.103	0	0:00:04.012689
200	[5]	2	0.158	0	0:00:04.011214
220	[7]	2	0.301	0	0:00:04.056638
240	[1]	-1	0.0	0	0:00:04.090756
260	[8]	3	0.0364	0	0:00:04.062104
280	[9]	-1	0.0	0	0:00:04.023245
300	[6]	6	0.079	1	0:00:03.995619
320	[3]	2	0.0229	0	0:00:04.030779
340	[2]	2	0.0648	1	0:00:04.044169
360	[9]	-1	0.0	0	0:00:04.036389
380	[6]	2	0.0418	0	0:00:04.040248
400	[9]	6	0.142	0	0:00:04.051367
420	[4]	2	0.0819	0	0:00:04.010386
440	[1]	6	0.0756	0	0:00:04.499271
460	[5]	2	0.264	0	0:00:04.021423
480	[8]	-1	0.0	0	0:00:04.007286
500	[4]	2	0.231	0	0:00:03.998921
520	[5]	2	0.0479	0	0:00:04.033502
540	[1]	-1	0.0	0	0:00:04.048344
560	[0]	2	0.0288	0	0:00:04.038919
580	[4

In [11]:
certify.perform_certification_test(denoiser_stab_classifier, dataset=(x_test, y_test), sigma=0.25,
							   outfile="certification_output/sigma_0.25_denoiser_stab")

idx	label	predict	radius	correct	time
0	[3]	2	0.117	0	0:00:04.600970
20	[7]	-1	0.0	0	0:00:04.047765
40	[4]	-1	0.0	0	0:00:03.965281
60	[7]	2	0.0528	0	0:00:03.957271
80	[8]	2	0.248	0	0:00:04.452398
100	[4]	3	0.149	0	0:00:03.969241
120	[8]	2	0.114	0	0:00:03.961567
140	[6]	2	0.000313	0	0:00:03.969164
160	[2]	2	0.0556	1	0:00:03.999055
180	[0]	2	0.106	0	0:00:04.071806
200	[5]	2	0.152	0	0:00:04.017439
220	[7]	2	0.306	0	0:00:04.023924
240	[1]	-1	0.0	0	0:00:03.982944
260	[8]	3	0.0346	0	0:00:03.964631
280	[9]	-1	0.0	0	0:00:03.954606
300	[6]	6	0.0821	1	0:00:04.005572
320	[3]	2	0.0283	0	0:00:03.984305
340	[2]	2	0.0696	1	0:00:03.965438
360	[9]	-1	0.0	0	0:00:03.983097
380	[6]	2	0.0392	0	0:00:04.001764
400	[9]	6	0.15	0	0:00:03.970584
420	[4]	2	0.0868	0	0:00:03.973104
440	[1]	6	0.0736	0	0:00:03.995256
460	[5]	2	0.272	0	0:00:03.982165
480	[8]	-1	0.0	0	0:00:04.030990
500	[4]	2	0.229	0	0:00:04.008516
520	[5]	2	0.0499	0	0:00:04.075229
540	[1]	-1	0.0	0	0:00:04.021760
560	[0]	2	0.027	0	0:00:03.952404
580	[4

In [ ]:
certification_result_with_denoiser = "certification_output/sigma_0.25_denoiser_mse"
certification_result_without_denoiser = "certification_output/sigma_0.25_no_denoiser"

analyze.plot_certified_accuracy_per_sigma_best_model(
    "denoiser_mse", 'With vs Without Denoiser', 1.0,
    methods=
        [analyze.Line(analyze.ApproximateAccuracy(certification_result_with_denoiser), "$\sigma = 0.12$")],
    label='With Denoiser',
    methods_base=
        [analyze.Line(analyze.ApproximateAccuracy(certification_result_without_denoiser), "$\sigma = 0.12$")], 
    label_base='Without Denoiser',
    sigmas=[0.25])

In [13]:
certification_result_with_denoiser = "certification_output/sigma_0.25_denoiser_stab"

analyze.plot_certified_accuracy_per_sigma_best_model(
    "denoiser_stab", 'With vs Without Denoiser', 1.0,
    methods=
        [analyze.Line(analyze.ApproximateAccuracy(certification_result_with_denoiser), "$\sigma = 0.12$")],
    label='With Denoiser',
    methods_base=
        [analyze.Line(analyze.ApproximateAccuracy(certification_result_without_denoiser), "$\sigma = 0.12$")], 
    label_base='Without Denoiser',
    sigmas=[0.25])